<a href="https://colab.research.google.com/github/fbelinchon/HuggingFace_ES/blob/main/Simple_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -qq install transformers[sentencepiece]

     |████████████████████████████████| 3.8 MB 25.7 MB/s 
     |████████████████████████████████| 67 kB 5.6 MB/s 
     |████████████████████████████████| 6.5 MB 59.9 MB/s 
     |████████████████████████████████| 895 kB 63.3 MB/s 
     |████████████████████████████████| 596 kB 52.7 MB/s 
     |████████████████████████████████| 1.2 MB 51.1 MB/s 


# Vamos a implementar un modelo de traducción usando diferentes métodos


## Método Simple: Pipeline

Es la forma más sencilla pero que nos ofrece menos personalizaciones.

Pasos:
* Importamos el objeto pipeline
* Le pasamos el tipo de tarea ("translation") y elmodelo a utilizar (Helsinki-NLP/opus-mt-es-en)

Al objeto resultante le podemos pasar una lista de sentencias en español y nos devolvera el listado de traducciones.

In [11]:
from transformers import pipeline

In [19]:
checkpoint = "Helsinki-NLP/opus-mt-es-en"
translator = pipeline("translation",model=checkpoint)

In [20]:
print(translator(["Hola este es mi perro","No encuentro donde he dejado mi coche"]))

[{'translation_text': 'Hey, this is my dog.'}, {'translation_text': "I can't find where I left my car."}]


## Método más detallado

En este caso vamos a realizar manualmente los pasos que se realizan con el objeto pipeline.

Pasos:
* Obtenemos el tokenizador asociado al modelo que vamos a utilizar.
* Obtenemos el modelo en sí (ya preentrenado).
* Convertimos nuestro texto a token.
* Los tokens del paso anterior los convertimos a números.
* El listado de números será la entrada al modelo. Los modelos de redes neuronales solamente pueden trabajar con números.
* La salida será un conjunto de números que tendremos que decodificar para obtener las sentencias traducidas

In [23]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint = "Helsinki-NLP/opus-mt-es-en"
tokenizer = AutoTokenizer.from_pretrained(checkpoint,use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [32]:
traducir=["Hola este es mi perro."]

batch = tokenizer(traducir,padding=True, truncation=True, return_tensors="pt")

print(batch)

{'input_ids': tensor([[2119,  104,   43,  155, 8693,    3,    0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}


Podemos observar que el resultado el resultadoes un diccionario con:
* inputs_id: La representación númerica de las palabras de la sentencia a traducir.
* attention_mask: máscara que nos permite gestionar varias sentencias de diferentes longitudes a la vez.

Vamos a comprobar que si decodificamos la misma secuencia de números nos da la frase en palabras.


In [33]:
tokenizer.decode([2119,104,43,155,8693,3,0])

'▁Hola▁este es mi▁perro.'

Ahora vamos a gestionar una lista de sentencias para ver el funcionamiento de padding=true y el significado de attention_mask

In [34]:
traducir=["Hola este es mi perro","No encuentro donde he dejado mi coche"]

batch = tokenizer(traducir,padding=True, truncation=True, return_tensors="pt")
print(batch)

{'input_ids': tensor([[ 2119,   104,    43,   155,  8693,     0, 65000, 65000],
        [   71,  6585,   542,    95,  7131,   155,  2638,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1]])}


Comos e puede apreciar las dos sentencias tienen distintas longitudes. Con padding=True indicamos que se añadan caracteres especiales al final de la sentencia más corta hasta que las dos sentencias tengan la misma longitud.

Si nos fijamos el attention_mask podemos ver que en la primera sentencia tenemos ceros en los dosúltimos token que son los que se han añadido para que tenga la misma longitud que la segunda sentencia. Esto indica al modelo que no utilice esos dos caracteres para calcular la atencción.

Vemos que si decodificamos la primera sentencias nos aparecen los caracteres de relleno al final <pad>.

In [35]:
tokenizer.decode([2119,104,43,155,8693,0, 65000, 65000])

'▁Hola▁este es mi▁perro<pad><pad>'

A continuación usamos el diccionario batch como entrada a nuestro modelo.

In [37]:
gen = model.generate(**batch)

print(gen)

tensor([[65000,  1068,     2,    58,    31,   125,  6913,     3,     0, 65000,
         65000, 65000, 65000],
        [65000,    33,    88,    20,    56,   634,   291,    33,  1263,   125,
           785,     3,     0]])


La salida se corresponde con los números asignados a la traducción de lassentencias.

Tenemos que pasarles el decodificador para obtener las palabras asociadas a esos números. El resultado es el mismo que el obtenido por el primer método.

In [41]:
tokenizer.batch_decode(gen)

['<pad> Hey, this is my dog.<pad><pad><pad><pad>',
 "<pad> I can't find where I left my car."]

In [42]:
tokenizer.batch_decode(gen, skip_special_tokens=True)

['Hey, this is my dog.', "I can't find where I left my car."]